#P4J library must be compiled and installed
#mexican hatt too

-march=x86-64
python setup.py build_ext --inplace
pip install .

In [1]:
import glob
dirs = glob.glob('/home/magdalena/pipeline/data_preprocessing/sixplusdets/2021_out/*')

In [2]:
print(dirs[:10])

['/home/magdalena/pipeline/data_preprocessing/sixplusdets/2021_out/array_ZTF21abyrvwg.pkl', '/home/magdalena/pipeline/data_preprocessing/sixplusdets/2021_out/array_ZTF18acejtqr.pkl', '/home/magdalena/pipeline/data_preprocessing/sixplusdets/2021_out/array_ZTF18ablrecx.pkl', '/home/magdalena/pipeline/data_preprocessing/sixplusdets/2021_out/array_ZTF21aaddejv.pkl', '/home/magdalena/pipeline/data_preprocessing/sixplusdets/2021_out/array_ZTF21abzjuxv.pkl', '/home/magdalena/pipeline/data_preprocessing/sixplusdets/2021_out/array_ZTF19acflkan.pkl', '/home/magdalena/pipeline/data_preprocessing/sixplusdets/2021_out/array_ZTF18abdvdtw.pkl', '/home/magdalena/pipeline/data_preprocessing/sixplusdets/2021_out/array_ZTF22aajuhud.pkl', '/home/magdalena/pipeline/data_preprocessing/sixplusdets/2021_out/array_ZTF21abfqpak.pkl', '/home/magdalena/pipeline/data_preprocessing/sixplusdets/2021_out/array_ZTF18acjkqun.pkl']


In [3]:
import pickle
from tqdm import tqdm

In [4]:
import pandas as pd
grab_oids = []
for dir in tqdm(dirs, total = len(dirs)):
    with (open(dir, "rb")) as openfile:
        load = pickle.load(openfile)
        grab_oids.append(load['oid'])
        

100%|██████████| 91377/91377 [00:44<00:00, 2074.66it/s]


In [5]:
oids_df = pd.DataFrame({'oid':grab_oids}, dtype='str') 
oids_df.dtypes

oid    object
dtype: object

In [6]:
ff_df = pd.read_parquet('/home/magdalena/pipeline/data_preprocessing/sixplusdets/partitions.parquet')

In [7]:
ff_oids = ff_df['oid']

In [ ]:
ff_oids

In [8]:
cross_contamination = pd.merge(ff_df,oids_df, on='oid', how='inner')['oid'].unique()
print(cross_contamination.shape)

(1111,)


In [9]:
oids_df = oids_df[~oids_df['oid'].isin(cross_contamination)]
oids_df

,oid
0,ZTF21abyrvwg
1,ZTF18acejtqr
2,ZTF18ablrecx
3,ZTF21aaddejv
4,ZTF21abzjuxv
...,...
91372,ZTF21abxnjyc
91373,ZTF21abotogu
91374,ZTF21abfqubp
91375,ZTF21abaausi


In [10]:
def create_single_train_val_split(df_oids: pd.DataFrame,split_frac:float):
    """Will create a train/validation split by splitting a pd dataframe of oids (single column dataframe where index is range of N and column contains the oids)

    Args:
        df_oids (pd.DataFrame): _description_
    """
    assert split_frac < 1
    assert df_oids.empty == False
    assert split_frac > 0
    validation = df_oids.sample(frac=0.2, random_state=42)  # for reproducibility
    #print(validation)
    train = df_oids.loc[~df_oids.index.isin(validation.index)]
    #train = train.values.flatten().tolist()
    #validation = validation.values.flatten().tolist()

    return train,validation

In [11]:
train,test = create_single_train_val_split(oids_df,0.2)

In [12]:
import h5py
import os
def create_h5_file(filename, data_dict):
    """
    Create an HDF5 file and save data from a dictionary.
    
    Parameters:
    filename (str): Name or path of the HDF5 file to create
    data_dict (dict): Dictionary containing the data to save
    """
    # Create directories if they don't exist
    directory = os.path.dirname(filename)
    if directory:
        os.makedirs(directory, exist_ok=True)
    
    with h5py.File(filename, 'w') as f:
        for key, value in data_dict.items():
            if not isinstance(value, np.ndarray):
                value = np.array(value)
            f.create_dataset(key, data=value)
    
    # Return the absolute path of the created file
    return os.path.abspath(filename)

In [13]:
trainvalkeys = {}
for i in range(5):
    train,val = create_single_train_val_split(oids_df,0.2)
    train = train.index.tolist()
    train.sort()
    val = val.index.tolist()
    val.sort()
   # print(train)
    #assert not np.any(np.in1d(train.values.flatten(), val.values.flatten()))
    trainvalkeys.update({f'train_{i}':train})
    trainvalkeys.update({f'validation_{i}':val})


In [14]:
from dataclasses import dataclass
class atat2h5:
    def __init__(self,out_path,input_dir):
        self.out = out_path
        self.input = input_dir
        pass 
    def get_oids():
        pass
    def establish_splits():
        pass
    def get_flux():
        pass 
    def get_mask():
        pass
    def get_time(): 
        pass
    def get_ft():
        pass
    def get_mask():
        pass
    def get_window_feats():
        pass
        

In [15]:
import numpy as npimport glob
dirs = glob.glob('/home/magd
file_path = create_h5_file('/home/magdalena/pipeline/data_preprocessing/sixplusdets/final_datasets/2021_no_contamination.h5', trainvalkeys)

In [ ]:
import h5py
with h5py.File('/home/magdalena/pipeline/data_preprocessing/sixplusdets/final_datasets/2021_no_contamination.h5', 'r') as f:
    # Print original dictionary keys
    
    # Print HDF5 root level keys
    print("HDF5 root level keys:", list(f.keys()))
    these_idx = f.get('train_0')[:100]
    print(f.get('md_cols'))
    

In [ ]:
import pandas as pd
grab_flux = [] 
for dir in tqdm(dirs, total = len(dirs)):
    with (open(dir, "rb")) as openfile:
        load = pickle.load(openfile) 
        grab_flux.append(load['flux'])
with h5py.File('/home/magdalena/pipeline/data_preprocessing/sixplusdets/final_datasets//2021_no_contamination.h5', 'r+') as f:
    # Print original dictionary keys
    f.create_dataset('flux',data =np.array(grab_flux).astype(float) )
    # Print HDF5 root level keys
    print("HDF5 root level keys:", list(f.keys()))
del grab_flux 


In [ ]:
grab_time = [] 
for dir in tqdm(dirs, total = len(dirs)):
    with (open(dir, "rb")) as openfile:
        load = pickle.load(openfile) 
        grab_time.append(load['time'])
with h5py.File('/home/magdalena/pipeline/data_preprocessing/sixplusdets/final_datasets//2021_no_contamination.h5', 'r+') as f:
    # Print original dictionary keys
    f.create_dataset('time',data =np.array(grab_time).astype(float) )
    # Print HDF5 root level keys
    print("HDF5 root level keys:", list(f.keys()))
del grab_time

In [ ]:
grab_mask = [] 
for dir in tqdm(dirs, total = len(dirs)):
    with (open(dir, "rb")) as openfile:
        load = pickle.load(openfile) 
        grab_mask.append(load['mask'])
with h5py.File('/home/magdalena/pipeline/data_preprocessing/sixplusdets/final_datasets//2021_no_contamination.h5', 'r+') as f:
    # Print original dictionary keys
    f.create_dataset('mask',data =np.array(grab_mask).astype(float) )
    # Print HDF5 root level keys
    print("HDF5 root level keys:", list(f.keys()))
del grab_mask


In [ ]:
grab_ft = [] 
for dir in tqdm(dirs, total = len(dirs)):
    with (open(dir, "rb")) as openfile:
        load = pickle.load(openfile) 
        feats = pd.DataFrame(load['ft_cols'].value.values)
        
        feats = feats.replace([np.inf, -np.inf], np.nan)
        grab_ft.append(feats.values)
with h5py.File('/home/magdalena/pipeline/data_preprocessing/sixplusdets/final_datasets//2021_no_contamination.h5', 'r+') as f:
    # Print original dictionary keys
    f.create_dataset('ft_cols',data =np.array(grab_ft).astype(float))
    # Print HDF5 root level keys
    print("HDF5 root level keys:", list(f.keys()))
del grab_ft

In [ ]:
grab_md = [] 
for dir in tqdm(dirs, total = len(dirs)):
    with (open(dir, "rb")) as openfile:
        load = pickle.load(openfile) 
        feats = pd.DataFrame(load['md_cols'].value.values)
        
        feats = feats.replace([np.inf, -np.inf], np.nan)
        grab_md.append(feats.values)
with h5py.File('/home/magdalena/pipeline/data_preprocessing/sixplusdets/final_datasets//2021_no_contamination.h5', 'r+') as f:
    # Print original dictionary keys
    f.create_dataset('md_cols',data =np.array(grab_md).astype(float))
    # Print HDF5 root level keys
    print("HDF5 root level keys:", list(f.keys()))
del grab_md

In [ ]:
asdfasdf

QT CREATE


In [10]:
grab_md = [] 
import numpy as np
for dir in tqdm(dirs, total = len(dirs)):
    with (open(dir, "rb")) as openfile:
        load = pickle.load(openfile) 
        feats = pd.DataFrame(load['md_cols'].value.values).astype(float)
        

        #feats = feats.fillna(-9999)
        
        feats = feats.replace([np.inf, -np.inf], np.nan)
    grab_md.append(feats.values.flatten())

    
from sklearn.preprocessing import QuantileTransformer
nan_mask = np.isnan(np.array(grab_md))

df = pd.DataFrame(grab_md)
qt = QuantileTransformer(
    n_quantiles=1000, random_state=0, output_distribution="uniform"
    )  
qt.fit(df[~nan_mask])
df = qt.transform(df.fillna(12345)) + 0.1
df[nan_mask] = 0.0
df = df.reshape(df.shape[0],df.shape[1],1).shape

100%|██████████| 91377/91377 [00:40<00:00, 2256.39it/s]
/home/magdalena/.local/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


In [11]:
df

(91377, 6, 1)

QT transform

In [20]:
import numpy as np
from sklearn.preprocessing import QuantileTransformer
from tqdm import tqdm
import joblib
import h5py


def create_ft_fold(out_dir, fold_name,seed):
    qt = QuantileTransformer(
    n_quantiles=1000, random_state=0, output_distribution="uniform"
    )   

    with h5py.File('/home/magdalena/pipeline/data_preprocessing/sixplusdets/final_datasets//2021_no_contamination.h5', 'r') as f:
        # Print original dictionary keys
        
        # Print HDF5 root level keys
        print("HDF5 root level keys:", list(f.keys()))
        these_idx = f.get(f'train_{seed}')
        print(these_idx)
        feats = f.get('ft_cols')[these_idx]
    feats = feats.reshape(feats.shape[0],feats.shape[1])
    nan_mask = np.isnan(np.array(feats))

    df = pd.DataFrame(feats)

    qt.fit(df[~nan_mask])
    #df = qt.transform(df.fillna(12345)) + 0.1
    #df[nan_mask] = 0.0
    #df = df.reshape(df.shape[0],df.shape[1],1)
    #feats = np.concatenate([feat for feat in collect_feats]) 
    #print(feats)
    print()
    
 
    joblib.dump(qt, f'{out_dir}/{fold_name}_qt-fold-{seed}.joblib')

In [24]:
import pandas as pd
def create_md_fold(out_dir, fold_name,seed):
    qt = QuantileTransformer(
    n_quantiles=1000, random_state=0, output_distribution="uniform"
    )   
    collect_feats = []

    with h5py.File('/home/magdalena/pipeline/data_preprocessing/sixplusdets/final_datasets//2021_no_contamination.h5', 'r') as f:
        # Print original dictionary keys
        
        # Print HDF5 root level keys
        print("HDF5 root level keys:", list(f.keys()))
        these_idx = f.get(f'train_{seed}')
        #print(these_idx)
        feats = f.get('md_cols')[these_idx] 
    feats = feats.reshape(feats.shape[0],feats.shape[1])
    nan_mask = np.isnan(np.array(feats))

    df = pd.DataFrame(feats)

    qt.fit(df[~nan_mask])
    print()
 
    joblib.dump(qt, f'{out_dir}/{fold_name}_qt-fold-{seed}.joblib')

In [25]:
for i in range(0,5):
    trans_feat = create_ft_fold('/home/magdalena/pipeline/data_preprocessing/sixplusdets/final_datasets/','ft',seed=i)
    trans_feat = create_md_fold('/home/magdalena/pipeline/data_preprocessing/sixplusdets/final_datasets/','md',seed=i)

HDF5 root level keys: ['flux', 'ft_cols', 'mask', 'md_cols', 'time', 'train_0', 'train_1', 'train_2', 'train_3', 'train_4', 'validation_0', 'validation_1', 'validation_2', 'validation_3', 'validation_4']
<HDF5 dataset "train_0": shape (72213,), type "<i8">

HDF5 root level keys: ['flux', 'ft_cols', 'mask', 'md_cols', 'time', 'train_0', 'train_1', 'train_2', 'train_3', 'train_4', 'validation_0', 'validation_1', 'validation_2', 'validation_3', 'validation_4']


/home/magdalena/.local/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,



HDF5 root level keys: ['flux', 'ft_cols', 'mask', 'md_cols', 'time', 'train_0', 'train_1', 'train_2', 'train_3', 'train_4', 'validation_0', 'validation_1', 'validation_2', 'validation_3', 'validation_4']
<HDF5 dataset "train_1": shape (72213,), type "<i8">

HDF5 root level keys: ['flux', 'ft_cols', 'mask', 'md_cols', 'time', 'train_0', 'train_1', 'train_2', 'train_3', 'train_4', 'validation_0', 'validation_1', 'validation_2', 'validation_3', 'validation_4']


/home/magdalena/.local/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,



HDF5 root level keys: ['flux', 'ft_cols', 'mask', 'md_cols', 'time', 'train_0', 'train_1', 'train_2', 'train_3', 'train_4', 'validation_0', 'validation_1', 'validation_2', 'validation_3', 'validation_4']
<HDF5 dataset "train_2": shape (72213,), type "<i8">

HDF5 root level keys: ['flux', 'ft_cols', 'mask', 'md_cols', 'time', 'train_0', 'train_1', 'train_2', 'train_3', 'train_4', 'validation_0', 'validation_1', 'validation_2', 'validation_3', 'validation_4']


/home/magdalena/.local/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,



HDF5 root level keys: ['flux', 'ft_cols', 'mask', 'md_cols', 'time', 'train_0', 'train_1', 'train_2', 'train_3', 'train_4', 'validation_0', 'validation_1', 'validation_2', 'validation_3', 'validation_4']
<HDF5 dataset "train_3": shape (72213,), type "<i8">

HDF5 root level keys: ['flux', 'ft_cols', 'mask', 'md_cols', 'time', 'train_0', 'train_1', 'train_2', 'train_3', 'train_4', 'validation_0', 'validation_1', 'validation_2', 'validation_3', 'validation_4']


/home/magdalena/.local/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,



HDF5 root level keys: ['flux', 'ft_cols', 'mask', 'md_cols', 'time', 'train_0', 'train_1', 'train_2', 'train_3', 'train_4', 'validation_0', 'validation_1', 'validation_2', 'validation_3', 'validation_4']
<HDF5 dataset "train_4": shape (72213,), type "<i8">

HDF5 root level keys: ['flux', 'ft_cols', 'mask', 'md_cols', 'time', 'train_0', 'train_1', 'train_2', 'train_3', 'train_4', 'validation_0', 'validation_1', 'validation_2', 'validation_3', 'validation_4']



/home/magdalena/.local/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
